# Uso de Metadados e Catálogo no Apache Iceberg

Este notebook demonstra como utilizar os ricos metadados e recursos de catálogo do Apache Iceberg para monitoramento, análise e governança de dados.

## Contexto e Objetivos:

### Importância dos Metadados:
- **Governança**: Controle e rastreabilidade de dados
- **Monitoramento**: Acompanhamento de performance e uso
- **Otimização**: Identificação de oportunidades de melhoria
- **Auditoria**: Compliance e histórico de mudanças
- **Troubleshooting**: Diagnóstico de problemas

### Recursos de Metadados do Iceberg:
- **Snapshots**: Histórico de versões da tabela
- **Manifests**: Informações sobre arquivos de dados
- **Files**: Detalhes de cada arquivo individual
- **History**: Cronologia de operações
- **Partitions**: Informações de particionamento
- **Refs**: Referências e branches

### Casos de Uso:
1. **Monitoramento de Performance**: Identificar gargalos
2. **Análise de Crescimento**: Acompanhar evolução dos dados
3. **Otimização de Queries**: Entender padrões de acesso
4. **Manutenção Preventiva**: Identificar necessidade de compactação
5. **Auditoria de Dados**: Rastrear mudanças e acessos

## Setup do Ambiente

In [4]:
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass

# Configuração do Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession

# Sessão Spark
spark = SparkSession.builder \
    .appName("IcebergMetadata") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

Warehouse configurado para: file:///home/tavares/warehouse


## 1. Criação de Tabela de Exemplo

Vamos criar uma tabela que será usada para demonstrar os recursos de metadados.

In [5]:
# Criando tabela de vendas
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas")

spark.sql("""
    CREATE TABLE hadoop_catalog.default.vendas (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
""")

DataFrame[]

## 2. Inserção de Dados para Gerar Metadados

Vamos inserir dados em várias operações para criar um histórico rico de metadados.

In [6]:
# Primeira inserção de dados
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (1, 'Produto A', 10, 15.5, DATE('2024-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2024-01-16')),
    (3, 'Produto C', 8, 30.0, DATE('2024-01-17'))
""")

print("Primeira inserção realizada")

Primeira inserção realizada


In [7]:
# Segunda inserção de dados
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (4, 'Produto D', 12, 25.0, DATE('2024-02-18')),
    (5, 'Produto E', 7, 18.5, DATE('2024-02-19'))
""")

print("Segunda inserção realizada")

Segunda inserção realizada


In [8]:
# Atualização de dados
spark.sql("""
    UPDATE hadoop_catalog.default.vendas 
    SET preco = preco * 1.1 
    WHERE id <= 2
""")

print("Atualização realizada")

Atualização realizada


## 3. Explorando Metadados de Snapshots

Os snapshots são fundamentais no Iceberg. Vamos explorar as informações disponíveis.

In [9]:
# Informações básicas dos snapshots
print("=== SNAPSHOTS DA TABELA ===")
spark.sql("""
    SELECT 
        snapshot_id,
        committed_at,
        operation,
        summary
    FROM hadoop_catalog.default.vendas.snapshots 
    ORDER BY committed_at
""").show(truncate=False)

# Contagem de snapshots
snapshot_count = spark.sql("SELECT COUNT(*) as total FROM hadoop_catalog.default.vendas.snapshots").collect()[0][0]
print(f"\nTotal de snapshots: {snapshot_count}")

=== SNAPSHOTS DA TABELA ===
+-------------------+-----------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|snapshot_id        |committed_at           |operation|summary                                                                                                                                                                                                                                                                                                                                 

## 4. Análise do Histórico da Tabela

O histórico fornece uma visão cronológica de todas as operações realizadas na tabela.

In [10]:
# Histórico completo da tabela
print("=== HISTÓRICO DA TABELA ===")
spark.sql("""
    SELECT 
        made_current_at,
        snapshot_id,
        parent_id,
        is_current_ancestor
    FROM hadoop_catalog.default.vendas.history 
    ORDER BY made_current_at
""").show(truncate=False)

# Análise temporal
print("\n=== ANÁLISE TEMPORAL ===")
spark.sql("""
    SELECT 
        DATE(made_current_at) as data_operacao,
        COUNT(*) as num_operacoes,
        MIN(made_current_at) as primeira_operacao,
        MAX(made_current_at) as ultima_operacao
    FROM hadoop_catalog.default.vendas.history 
    GROUP BY DATE(made_current_at)
    ORDER BY data_operacao
""").show()

=== HISTÓRICO DA TABELA ===
+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-11-02 23:50:26.548|3956159667302237607|null               |true               |
|2025-11-02 23:50:29.171|5556644385757808960|3956159667302237607|true               |
|2025-11-02 23:50:35.171|8599238022227158035|5556644385757808960|true               |
+-----------------------+-------------------+-------------------+-------------------+


=== ANÁLISE TEMPORAL ===
+-------------+-------------+--------------------+--------------------+
|data_operacao|num_operacoes|   primeira_operacao|     ultima_operacao|
+-------------+-------------+--------------------+--------------------+
|   2025-11-02|            3|2025-11-02 23:50:...|2025-11-02 23:50:...|
+-------------+-------------+--------------------+-----

## 5. Informações Detalhadas dos Arquivos

Vamos analisar os arquivos de dados e suas características.

In [11]:
# Informações dos arquivos de dados
print("=== ARQUIVOS DE DADOS ===")
spark.sql("""
    SELECT 
        file_path,
        file_format,
        record_count,
        file_size_in_bytes,
        ROUND(file_size_in_bytes / 1024.0, 2) as file_size_kb,
        column_sizes,
        value_counts,
        null_value_counts
    FROM hadoop_catalog.default.vendas.files
""").show(truncate=False)

# Estatísticas agregadas dos arquivos
print("\n=== ESTATÍSTICAS DOS ARQUIVOS ===")
spark.sql("""
    SELECT 
        COUNT(*) as total_arquivos,
        SUM(record_count) as total_registros,
        AVG(record_count) as media_registros_por_arquivo,
        MIN(record_count) as min_registros,
        MAX(record_count) as max_registros,
        SUM(file_size_in_bytes) as tamanho_total_bytes,
        ROUND(AVG(file_size_in_bytes), 2) as tamanho_medio_bytes
    FROM hadoop_catalog.default.vendas.files
""").show()

=== ARQUIVOS DE DADOS ===
+-------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+------------+---------------------------------------------+----------------------------------------+----------------------------------------+
|file_path                                                                                                    |file_format|record_count|file_size_in_bytes|file_size_kb|column_sizes                                 |value_counts                            |null_value_counts                       |
+-------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+------------+---------------------------------------------+----------------------------------------+----------------------------------------+
|file:/home/tavares/warehouse/default/vendas/data/00015-7-f5f70203-8385-4de5-b62f-6

## 6. Análise de Manifests

Os manifests contêm metadados sobre os arquivos de dados e são essenciais para a performance.

In [12]:
# Informações dos manifests
print("=== MANIFESTS ===")
spark.sql("""
    SELECT 
        path,
        length,
        partition_spec_id,
        added_snapshot_id,
        added_data_files_count,
        existing_data_files_count,
        deleted_data_files_count
    FROM hadoop_catalog.default.vendas.manifests
""").show(truncate=False)

# Estatísticas dos manifests
print("\n=== ESTATÍSTICAS DOS MANIFESTS ===")
spark.sql("""
    SELECT 
        COUNT(*) as total_manifests,
        SUM(added_data_files_count) as total_arquivos_adicionados,
        SUM(existing_data_files_count) as total_arquivos_existentes,
        SUM(deleted_data_files_count) as total_arquivos_deletados,
        AVG(length) as tamanho_medio_manifest
    FROM hadoop_catalog.default.vendas.manifests
""").show()

=== MANIFESTS ===
+-------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+
|path                                                                                             |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|
+-------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+
|file:/home/tavares/warehouse/default/vendas/metadata/7c25a193-a7c6-483f-8190-0814dd4cf0b9-m1.avro|6945  |0                |8599238022227158035|2                     |0                        |0                       |
|file:/home/tavares/warehouse/default/vendas/metadata/6b49ef22-e9d4-48fe-8fd8-e9f496f0d34c-m0.avro|6928  |

## 7. Informações do Schema e Particionamento

Vamos explorar informações sobre o schema da tabela e particionamento.

In [13]:
# Schema da tabela
print("=== SCHEMA DA TABELA ===")
spark.sql("DESCRIBE EXTENDED hadoop_catalog.default.vendas").show(truncate=False)

# Informações de particionamento (se houver)
print("\n=== INFORMAÇÕES DE PARTICIONAMENTO ===")
try:
    spark.sql("SELECT * FROM hadoop_catalog.default.vendas.partitions").show()
except:
    print("Tabela não está particionada")

# Propriedades da tabela
print("\n=== PROPRIEDADES DA TABELA ===")
spark.sql("SHOW TBLPROPERTIES hadoop_catalog.default.vendas").show(truncate=False)

=== SCHEMA DA TABELA ===
+----------------------------+---------------------------------------------+-------+
|col_name                    |data_type                                    |comment|
+----------------------------+---------------------------------------------+-------+
|id                          |int                                          |       |
|produto                     |string                                       |       |
|quantidade                  |int                                          |       |
|preco                       |double                                       |       |
|data_venda                  |date                                         |       |
|                            |                                             |       |
|# Partitioning              |                                             |       |
|Not partitioned             |                                             |       |
|                            |          

## 8. Análise de Performance e Otimização

Usando metadados para identificar oportunidades de otimização.

In [14]:
# Análise de fragmentação de arquivos
print("=== ANÁLISE DE FRAGMENTAÇÃO ===")
spark.sql("""
    SELECT 
        CASE 
            WHEN record_count < 1000 THEN 'Muito Pequeno (<1K)'
            WHEN record_count < 10000 THEN 'Pequeno (1K-10K)'
            WHEN record_count < 100000 THEN 'Médio (10K-100K)'
            ELSE 'Grande (>100K)'
        END as categoria_tamanho,
        COUNT(*) as num_arquivos,
        SUM(record_count) as total_registros,
        ROUND(AVG(record_count), 0) as media_registros,
        ROUND(SUM(file_size_in_bytes) / 1024.0 / 1024.0, 2) as tamanho_total_mb
    FROM hadoop_catalog.default.vendas.files
    GROUP BY 
        CASE 
            WHEN record_count < 1000 THEN 'Muito Pequeno (<1K)'
            WHEN record_count < 10000 THEN 'Pequeno (1K-10K)'
            WHEN record_count < 100000 THEN 'Médio (10K-100K)'
            ELSE 'Grande (>100K)'
        END
    ORDER BY categoria_tamanho
""").show()

# Recomendações baseadas em metadados
file_count = spark.sql("SELECT COUNT(*) FROM hadoop_catalog.default.vendas.files").collect()[0][0]
avg_records = spark.sql("SELECT AVG(record_count) FROM hadoop_catalog.default.vendas.files").collect()[0][0]

print(f"\n=== RECOMENDAÇÕES ===")
if file_count > 10:
    print(f"⚠️  Muitos arquivos ({file_count}): Considere compactação")
if avg_records < 1000:
    print(f"⚠️  Arquivos pequenos (média: {avg_records:.0f} registros): Compactação recomendada")
if file_count <= 5 and avg_records > 1000:
    print("✅ Estrutura de arquivos otimizada")

=== ANÁLISE DE FRAGMENTAÇÃO ===
+-------------------+------------+---------------+---------------+----------------+
|  categoria_tamanho|num_arquivos|total_registros|media_registros|tamanho_total_mb|
+-------------------+------------+---------------+---------------+----------------+
|Muito Pequeno (<1K)|           5|              5|            1.0|            0.01|
+-------------------+------------+---------------+---------------+----------------+


=== RECOMENDAÇÕES ===
⚠️  Arquivos pequenos (média: 1 registros): Compactação recomendada


## 9. Monitoramento de Crescimento de Dados

Análise temporal do crescimento dos dados usando metadados.

In [15]:
# Crescimento de dados ao longo do tempo
print("=== CRESCIMENTO DE DADOS ===")
spark.sql("""
    SELECT 
        s.committed_at,
        s.operation,
        COALESCE(s.summary['added-records'], '0') as registros_adicionados,
        COALESCE(s.summary['deleted-records'], '0') as registros_deletados,
        COALESCE(s.summary['changed-partition-count'], '0') as particoes_alteradas
    FROM hadoop_catalog.default.vendas.snapshots s
    ORDER BY s.committed_at
""").show(truncate=False)

# Evolução do tamanho da tabela
print("\n=== EVOLUÇÃO DO TAMANHO ===")
current_size = spark.sql("""
    SELECT 
        COUNT(*) as total_registros,
        ROUND(SUM(file_size_in_bytes) / 1024.0 / 1024.0, 2) as tamanho_mb
    FROM hadoop_catalog.default.vendas.files
""").show()

=== CRESCIMENTO DE DADOS ===
+-----------------------+---------+---------------------+-------------------+-------------------+
|committed_at           |operation|registros_adicionados|registros_deletados|particoes_alteradas|
+-----------------------+---------+---------------------+-------------------+-------------------+
|2025-11-02 23:50:26.548|append   |3                    |0                  |1                  |
|2025-11-02 23:50:29.171|append   |2                    |0                  |1                  |
|2025-11-02 23:50:35.171|overwrite|2                    |2                  |1                  |
+-----------------------+---------+---------------------+-------------------+-------------------+


=== EVOLUÇÃO DO TAMANHO ===
+---------------+----------+
|total_registros|tamanho_mb|
+---------------+----------+
|              5|      0.01|
+---------------+----------+



## 10. Consultas Avançadas de Metadados

Exemplos de consultas mais complexas para análise avançada.

In [16]:
# Análise de eficiência de operações
print("=== EFICIÊNCIA DE OPERAÇÕES ===")
spark.sql("""
    SELECT 
        operation,
        COUNT(*) as num_operacoes,
        AVG(CAST(COALESCE(summary['added-records'], '0') AS INT)) as media_registros_por_op
    FROM hadoop_catalog.default.vendas.snapshots
    GROUP BY operation
    ORDER BY num_operacoes DESC
""").show()

# Identificar arquivos com estatísticas interessantes
print("\n=== ARQUIVOS COM CARACTERÍSTICAS ESPECIAIS ===")
spark.sql("""
    SELECT 
        file_path,
        record_count,
        ROUND(file_size_in_bytes / record_count, 2) as bytes_por_registro,
        null_value_counts
    FROM hadoop_catalog.default.vendas.files
    WHERE record_count > 0
    ORDER BY bytes_por_registro DESC
""").show(truncate=False)

# Análise de densidade de dados
print("\n=== DENSIDADE DE DADOS ===")
spark.sql("""
    SELECT 
        ROUND(AVG(file_size_in_bytes / record_count), 2) as bytes_medio_por_registro,
        ROUND(MIN(file_size_in_bytes / record_count), 2) as min_bytes_por_registro,
        ROUND(MAX(file_size_in_bytes / record_count), 2) as max_bytes_por_registro
    FROM hadoop_catalog.default.vendas.files
    WHERE record_count > 0
""").show()

=== EFICIÊNCIA DE OPERAÇÕES ===
+---------+-------------+----------------------+
|operation|num_operacoes|media_registros_por_op|
+---------+-------------+----------------------+
|   append|            2|                   2.5|
|overwrite|            1|                   2.0|
+---------+-------------+----------------------+


=== ARQUIVOS COM CARACTERÍSTICAS ESPECIAIS ===
+-------------------------------------------------------------------------------------------------------------+------------+------------------+----------------------------------------+
|file_path                                                                                                    |record_count|bytes_por_registro|null_value_counts                       |
+-------------------------------------------------------------------------------------------------------------+------------+------------------+----------------------------------------+
|file:/home/tavares/warehouse/default/vendas/data/00015-7-f5f70203-838

## Resumo dos Recursos de Metadados

### Tabelas de Metadados Disponíveis:

1. **`table.snapshots`**: Histórico de versões da tabela
   - snapshot_id, committed_at, operation, summary
   - Útil para: auditoria, rollback, análise temporal

2. **`table.history`**: Cronologia de mudanças
   - made_current_at, snapshot_id, parent_id
   - Útil para: rastreamento de mudanças, genealogia

3. **`table.files`**: Informações detalhadas dos arquivos
   - file_path, record_count, file_size_in_bytes, column_sizes
   - Útil para: otimização, compactação, análise de performance

4. **`table.manifests`**: Metadados dos manifests
   - path, length, added_data_files_count
   - Útil para: diagnóstico de performance, otimização de metadados

5. **`table.partitions`**: Informações de particionamento
   - partition, record_count, file_count
   - Útil para: análise de distribuição, otimização de queries

### Casos de Uso Práticos:

- **Monitoramento**: Dashboards de crescimento e uso
- **Otimização**: Identificação de necessidade de compactação
- **Troubleshooting**: Diagnóstico de problemas de performance
- **Governança**: Auditoria e compliance
- **Planejamento**: Previsão de crescimento e recursos

### Benefícios dos Metadados Ricos:

- **Visibilidade**: Transparência completa sobre os dados
- **Automação**: Possibilidade de automação baseada em métricas
- **Otimização**: Decisões informadas sobre manutenção
- **Confiabilidade**: Maior confiança na qualidade dos dados
- **Eficiência**: Operações mais eficientes baseadas em dados reais